In [320]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
%matplotlib inline
pd.set_option('display.max_colwidth',999)

### Variable List
df --> DataFrame (1778,10) with original data

df1 --> DataFrame (1612,7) with unused_cols dropped, and null values from Formatted Result dropped

unused_cols -->  DataFrame (1778,3) columns Pukie, Notes and Description

no_work--> DataFrame (63,7) no work was performed - Warmsups and Cooldowns, not needed for performance calcs

lifts --> DataFrame (45,7) lifts only, work completed but no time.  These will be used to calculate volume abilities. 



In [321]:
df = pd.read_csv('data.csv')

,Date,Workout,Result,Prescribed,Pukie,Work performed,Work time,Formatted Result,Notes,Description
0,2017-04-04,"5 RFT: Power Cleans, Walking Lunges, and Thrusters",399000.000000,True,False,42039,399000.0,6 mins 39 secs,Went out too hot,"5 rounds of:\n12 Power Cleans, 65 lbs\n6 Walking Lunges, 65 lbs\n3 Thrusters, 65 lbs"
1,2017-04-02,Cindy,11.133000,True,False,88204,1200000.0,11 rounds + 2 Pull-ups | 332 reps,NaN,20:00 AMRAP:\n5 Pull-ups\n10 Push-ups\n15 Air Squats
2,2017-03-28,21-15-9: Push Press and Box Jumps,419000.000000,True,False,21197,419000.0,6 mins 59 secs,"Step ups, got a set of 13 pp on the first round","21-15-9 reps of:\nPush Press, 95 lbs\nBox Jump, 20 in"
3,2017-03-28,Deadlift 4x3,1115.837230,True,False,6207,NaN,"2460 lbs | 205 lbs, 205 lbs, 205 lbs, and 205 lbs",NaN,Sets\n3 Deadlifts | 205 lbs\n3 Deadlifts | 205 lbs\n3 Deadlifts | 205 lbs\n3 Deadlifts | 205 lbs
4,2017-03-26,Split Jerk 1 Rep,77.110703,True,False,300,NaN,170 lbs | 170 lbs,Went for 175 but missed,Sets\n1 Split Jerk | 170 lbs
5,2017-03-26,"Row : 3x 1 km, rest 3 mins",771500.000000,True,False,93741,771500.0,12 mins 51.5 secs,NaN,"Intervals : rest 3 mins\nRow, 1 km | 4:11.9\nRow, 1 km | 4:19.3\nRow, 1 km | 4:20.3"
6,2017-03-24,CrossFit Games Open 17.5,982000.000000,True,False,66793,982000.0,16 mins 22 secs,"All thrusters unbroken, 5 of the rounds of dubs unbroken","10 rounds of:\n9 Thrusters, 65 lbs\n35 Double Unders"
7,2017-03-22,Every 1:30 for 10:30: Front Squat,857.289579,True,False,5848,630000.0,1890 lbs,NaN,Every 1:30 for 10:30:\n2 Front Squats | 135 lbs\n2 Front Squats | 135 lbs\n2 Front Squats | 135 lbs\n2 Front Squats | 135 lbs\n2 Front Squats | 135 lbs\n2 Front Squats | 135 lbs\n2 Front Squats | 135 lbs
8,2017-03-22,AMRAP 5 mins: Squat Cleans and AbMat Sit-ups,3.810000,True,False,29387,300000.0,3 rounds + 7 Squat Cleans + 13 AbMat Sit-ups | 104 reps,NaN,"5:00 AMRAP:\n7 Squat Cleans, 75 lbs\n21 AbMat Sit-ups"
9,2017-03-22,AMRAP 5 mins: Hanging Knee Raises and Push Press,3.000000,True,False,2780,300000.0,3 rounds | 84 reps,NaN,"5:00 AMRAP:\n21 Hanging Knee Raises\n7 Push Press, 75 lbs"


In [335]:
df_wo = pd.read_csv('df_wo.csv')

In [322]:
games_dis = []
for row in df.Workout:
    if 'Games' in row:
        games_dis.append(list(df.loc[df.Workout == row,'Description'])[0])

def add_games_dis(cell):
    if 'Games' in cell:
        return games_dis.pop(0)
    else:
        return cell    

df.Workout = df.Workout.apply(lambda x: add_games_dis(x))

In [323]:
unused_cols = df[['Pukie','Notes']]
unused_cols.shape

(1778, 3)

In [324]:
df1 = df.drop(['Pukie','Notes'], axis=1)

In [325]:
df1 = df1.dropna(subset=['Formatted Result'])
df1.shape

(1612, 8)

In [327]:
no_work = df1.loc[(df['Formatted Result'] == 'Completed') & (df['Work performed'] == 0), :]
no_work.shape

(63, 8)

In [328]:
lifts = df1.loc[(df1['Work time'].isnull()) & (df1['Formatted Result'].str.endswith('| Completed') | df1['Formatted Result'].str.startswith('Completed |')), :]

In [330]:
df1['workout_name_length'] = df1['Workout'].apply(lambda x: len(x))

In [331]:
bench = df1.loc[df1['workout_name_length'] < 10,:].groupby('Workout')['workout_name_length'].mean()

In [332]:
bench = [x for x in bench.index]

In [333]:
workouts = [x for x in df1.Workout]
work_string = ' '.join(workouts)

In [336]:
bench_dict = {}
for x in df_wo.benchmark:
    bench_dict[x] = df_wo.loc[df_wo.benchmark == x, 'move_list']


In [337]:
movements = pd.read_csv('movements.csv', index_col='Unnamed: 0')
add_moves = [{'movement':'double-under','frequency':153,'move_class':'Monostructural'},{'movement':'squat','frequency':63,'move_class':'Gymnastics'}]
am = pd.DataFrame(add_moves)
new_move = pd.concat([movements,am])


In [338]:
move_dict = movements.movement.to_dict()
res = dict((v,k) for k,v in move_dict.iteritems())

In [340]:
for x in workouts:
    if x.lower() in bench_dict:
        y = bench_dict[x.lower()].tolist()[0].split("'")[1::2]
        workouts[workouts.index(x)] = x + ':'+ ' '.join(y)
    if 'Games' in x:
        workouts[workouts.index(x)] = games_dis.pop(0) 


In [341]:
move_in_work = []
for w in workouts:
    moves = []
    if w in bench:
        moves.append(w)
    for m in new_move.movement:
        if m in w.lower():
            moves.append(m)
    move_in_work.append(moves)
    
bar = [', '.join(x) for x in move_in_work]


In [342]:
move_in_bench = []
for w in df_wo.move_list:
    moves = []
    for m in movements.movement:
        if m in w.lower():
            moves.append(m)
    move_in_bench.append(moves)
    
foo = [','.join(x) for x in move_in_bench]
len(foo)

147

In [343]:
df_wo['movements'] = move_in_bench


In [344]:
df1['Workout'] = workouts

In [345]:
df1['Movements'] = bar

In [346]:
df1['power'] = df['Work performed'] / (df['Work time']/1000)

In [348]:
df1 = df1.loc[(df1['Work performed']>1000) & (df1['Work time'] < 5000000),:].copy()

In [352]:
from sklearn.feature_extraction.text import CountVectorizer

In [353]:
X = df1['Movements']

vect = CountVectorizer(vocabulary=res)
#token_pattern =u'(?u)\\b[a-zA-Z_]{2,}\\b' 
#(?u)\b([a-zA-Z-]+?)(s\b|\b)\b
X_vect = vect.fit_transform(X)

In [354]:
len(X)

741

In [362]:
X_vect

<741x428 sparse matrix of type '<type 'numpy.int64'>'
	with 703 stored elements in Compressed Sparse Row format>

In [356]:
dtm = pd.DataFrame(X_vect.toarray(), columns = vect.get_feature_names())

In [360]:
dtm = pd.concat([df1[['Work performed','Work time','power','Movements']], dtm], axis=1, join='left')
dtm.shape

(486, 436)

In [192]:
len(df1.loc[df1['Workout'].apply(len)<20,:])

52

In [359]:
dtm.to_csv('model_wo.csv')